In [2]:
import zipfile
with zipfile.ZipFile("/home/siu856542507/zarif/FederatedMeta/EuroSAT.zip","r") as zip_ref:
    zip_ref.extractall("/home/siu856542507/zarif/FederatedMeta")

In [3]:
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms as transforms
import json
import os
import torch
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets,models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
batch_size = 4
client_aug = 2
augment= False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()

import warnings
warnings.filterwarnings('ignore')

In [5]:

#   transforms.Normalize([0.3444, 0.3803, 0.4078], [0.2027, 0.1369, 0.1156])
#         ]) weights="IMAGENET1K_V2"

In [6]:
train_dataset=datasets.ImageFolder(root="/home/siu856542507/zarif/FederatedMeta/2750")


In [7]:
from torch.utils.data import random_split
from torch.utils.data import Dataset

random_seed = 22
torch.manual_seed(random_seed);
# val_size = 2500
test_size = 2500
train_size = len(train_dataset)  - test_size

train_ds, test_ds  = random_split(train_dataset, [train_size, test_size])
len(train_ds), len(test_ds)

(24500, 2500)

In [8]:
train_transform = transforms.Compose([transforms.Resize(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.3444, 0.3803, 0.4078], [0.2027, 0.1369, 0.1156])
                                 ])

test_transform = transforms.Compose([transforms.Resize(32),
                                 transforms.ToTensor(),
                                transforms.Normalize([0.3444, 0.3803, 0.4078], [0.2027, 0.1369, 0.1156])
                                    ])
class ApplyTransform(Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform
        # yes, you don't need these 2 lines below :(
        if transform is None and target_transform is None:
            print("Am I a joke to you? :)")

    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

    def __len__(self):
        return len(self.dataset)

train_ds = ApplyTransform(train_ds, train_transform)
test_ds = ApplyTransform(test_ds,test_transform)


In [9]:
# class ConvNeuralNet(nn.Module):
# 	#  Determine what layers and their order in CNN object 
#     def __init__(self):
#         super(ConvNeuralNet, self).__init__()
#         self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
#         self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
#         self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
#         self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
#         self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.fc1 = nn.Linear(1600, 128)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(128, 14)
    
#     # Progresses data across layers    
#     def forward(self, x):
#         out = self.conv_layer1(x)
#         out = self.conv_layer2(out)
#         out = self.max_pool1(out)
        
#         out = self.conv_layer3(out)
#         out = self.conv_layer4(out)
#         out = self.max_pool2(out)
                
#         out = out.reshape(out.size(0), -1)
        
#         out = self.fc1(out)
#         out = self.relu1(out)
#         out = self.fc2(out)
#         return out
    
# model = ConvNeuralNet().to(device)

In [10]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

# transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),


def non_iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle, shuffle_digits=False):
    assert(nb_nodes>0 and nb_nodes<=10)

    digits=torch.arange(10) if shuffle_digits==False else torch.randperm(10, generator=torch.Generator().manual_seed(0))
    digits2=torch.arange(3) if shuffle_digits==False else torch.randperm(10, generator=torch.Generator().manual_seed(0))

    digits = torch.cat((digits,digits2))

    print(digits)
   
    digits_split=list()
    i=0
    for n in range(nb_nodes, 0, -1):
        inc=int((10-i)/n)
        digits_split.append(digits[i:i+3])
        print
        i+=inc
        print(inc)
        print(digits_split)
    
    print(digits_split)
    # load and shuffle nb_nodes*n_samples_per_node from the dataset
    loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=nb_nodes*n_samples_per_node,
                                        shuffle=shuffle)
    dataiter = iter(loader)
    images_train_mnist, labels_train_mnist = next(dataiter)

    data_splitted=list()
    for i in range(nb_nodes):
        idx=torch.stack([y_ == labels_train_mnist for y_ in digits_split[i]]).sum(0).bool() # get indices for the digits
        data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(images_train_mnist[idx], labels_train_mnist[idx]), batch_size=batch_size, shuffle=shuffle))
    print(data_splitted)
    return data_splitted

def iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle):
    # load and shuffle n_samples_per_node from the dataset
    loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=n_samples_per_node,
                                        shuffle=shuffle)
    dataiter = iter(loader)

    data_splitted=list()

    for _ in range(nb_nodes):
        data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(*(next(dataiter))), batch_size=batch_size, shuffle=shuffle))

    return data_splitted


def  get_violence(type, n_samples_train, n_samples_test, n_clients, batch_size, shuffle,train_dataset , test_dataset):

        if type=="iid":
            train=iid_split(train_dataset, n_clients, n_samples_train, batch_size, shuffle)
            test=iid_split(test_dataset, n_clients, n_samples_test, batch_size, shuffle)
        elif type=="non_iid":
            train=non_iid_split(train_dataset, n_clients, n_samples_train, batch_size, shuffle)
            test=non_iid_split(test_dataset, n_clients, n_samples_test, batch_size, shuffle)
        else:
            train=[]
            test=[]

        return train, test
    
def  get_MNIST(type="iid", n_samples_train=200, n_samples_test=100, n_clients=3, batch_size=25, shuffle=True):
    dataset_loaded_train = datasets.MNIST(
            root="./data",
            train=True,
            download=True,
            transform=transforms.ToTensor()
    )
    dataset_loaded_test = datasets.MNIST(
            root="./data",
            train=False,
            download=True,
            transform=transforms.ToTensor()
    )

    if type=="iid":
        train=iid_split(dataset_loaded_train, n_clients, n_samples_train, batch_size, shuffle)
        test=iid_split(dataset_loaded_test, n_clients, n_samples_test, batch_size, shuffle)
    elif type=="non_iid":
        train=non_iid_split(dataset_loaded_train, n_clients, n_samples_train, batch_size, shuffle)
        test=non_iid_split(dataset_loaded_test, n_clients, n_samples_test, batch_size, shuffle)
    else:
        train=[]
        test=[]

    return train, test
def plot_samples(data, channel:int, title=None, plot_name="", n_examples =20):

    n_rows = int(n_examples / 5)
    plt.figure(figsize=(1* n_rows, 1*n_rows))
    if title: plt.suptitle(title)
    X, y= data
    for idx in range(n_examples):
        
        ax = plt.subplot(n_rows, 5, idx + 1)

        image = 255 - X[idx, channel].view((64,64))
        ax.imshow(image, cmap='gist_gray')
        ax.axis("off")

    if plot_name!="":plt.savefig(f"plots/"+plot_name+".png")

    plt.tight_layout()

In [20]:
mnist_iid_train_dls, mnist_iid_test_dls = get_violence("iid",
    n_samples_train =500, n_samples_test=100, n_clients =10, 
    batch_size = batch_size, shuffle =True,train_dataset=train_ds,test_dataset=test_ds)

In [16]:

# mnist_iid_train_dls, mnist_iid_test_dls = get_MNIST("iid",
#     n_samples_train =500, n_samples_test=100, n_clients =10, 
#     batch_size =batch_size, shuffle =True)

In [17]:
# from torchsummary import summary

# model = models.resnet18(weights='IMAGENET1K_V1')

# for param in model.parameters():
#     param.requires_grad = False

# num_features = model.fc.in_features
# model.fc = nn.Sequential(
#     nn.Linear(num_features, 256),
#     nn.ReLU(),
#     nn.Linear(256, 128),
#     nn.ReLU(),
#     nn.Linear(128, 64),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
#     nn.Linear(64, NUM_CLASSES)
# )

# model.classifier[6] = nn.Sequential(
#                       nn.Linear(4096, 2048), 
#                       nn.ReLU(), 
#                       nn.Dropout(0.4),
#                       nn.Linear(2048, 1024),
#                       nn.ReLU(),
#                       nn.Dropout(0.2),
#                       nn.Linear(1024, 512),
#                       nn.ReLU(),
#                       nn.Dropout(0.1),
#                       nn.Linear(512,NUM_CLASSES),                   
#                       nn.LogSoftmax(dim=1))

# model.classifier

# # Transfer the model to device
# model = model.to(device)
# # Transfer the model to device


In [27]:
model =models.resnet18(weights='IMAGENET1K_V1')

# for param in model.parameters():
#     param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

# # Transfer the model to device
model = model.to(device)
# # Transfer the model to device


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/siu856542507/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.8MB/s]


In [28]:
# class MnistResNet(nn.Module):
#   def __init__(self, in_channels=3):
#     super(MnistResNet, self).__init__()

#     # Load a pretrained resnet model from torchvision.models in Pytorch
#     self.model = models.resnet50(pretrained=True)

#     # Change the input layer to take Grayscale image, instead of RGB images. 
#     # Hence in_channels is set as 1 or 3 respectively
#     # original definition of the first layer on the ResNet class
#     # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#     self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
#     # Change the output layer to output 10 classes instead of 1000 classes
#     num_ftrs = self.model.fc.in_features
#     self.model.fc = nn.Linear(num_ftrs, 14)

#   def forward(self, x):
#     return self.model(x)


# my_resnet = MnistResNet()

# # input = torch.randn((16,1,244,244))
# # output = my_resnet(input)

# model = MnistResNet().to(device)


In [29]:
def loss_classifier(predictions,labels):
    
#     m = nn.LogSoftmax(dim=1)
    loss = nn.CrossEntropyLoss()
    
    return loss(predictions ,labels)


def loss_dataset(model, dataset, loss_f):
    """Compute the loss of `model` on `dataset`"""
    loss=0
    
    for idx,(features,labels) in enumerate(dataset):
        features = features.to(device)
        labels=labels.to(device)
        predictions= model(features)
        loss+=loss_f(predictions,labels)
    
    loss/=idx+1
    return loss


def accuracy_dataset(model, dataset):
    """Compute the accuracy of `model` on `dataset`"""
    
    correct=0
    
    for features,labels in iter(dataset):
        features = features.to(device)
        labels=labels.to(device)
        predictions= model(features)
        
        _,predicted=predictions.max(1,keepdim=True)
        
        correct+=torch.sum(predicted.view(-1,1)==labels.view(-1, 1)).item()
        
    accuracy = 100*correct/len(dataset.dataset)
        
    return accuracy


def train_step(model, model_0, mu:int,scheduler, optimizer, train_data, loss_f):
    """Train `model` on one epoch of `train_data`"""
    
    total_loss=0
    
    for idx, (features,labels) in enumerate(train_data):
        
        optimizer.zero_grad()
        features = features.to(device)
        predictions= model(features)
        labels=labels.to(device)
        loss=loss_f(predictions,labels)
        loss+=mu/2*difference_models_norm_2(model,model_0)
        total_loss+=loss
        
        loss.backward()
    
        optimizer.step()
    scheduler.step()
   
    return total_loss/(idx+1)



def local_learning(model, mu:float, optimizer,exp_lr_scheduler, train_data, epochs:int, loss_f
):
    
    model_0=deepcopy(model)
    
    for e in range(epochs):
        local_loss=train_step(model,model_0,mu,exp_lr_scheduler,optimizer,train_data,loss_f)
        
    return float(local_loss.detach().cpu().numpy())


def difference_models_norm_2(model_1, model_2):
    """Return the norm 2 difference between the two model parameters
    """
    
    tensor_1=list(model_1.parameters())
    tensor_2=list(model_2.parameters())
    
    norm=sum([torch.sum((tensor_1[i]-tensor_2[i])**2) 
        for i in range(len(tensor_1))])
    
    return norm


def set_to_zero_model_weights(model):
    """Set all the parameters of a model to 0"""

    for layer_weigths in model.parameters():
        layer_weigths.data.sub_(layer_weigths.data)

In [30]:
def average_models(model, clients_models_hist:list , weights:list):

    """Creates the new model of a given iteration with the models of the other

    clients"""
    
    new_model=deepcopy(model)
    set_to_zero_model_weights(new_model)

    for k,client_hist in enumerate(clients_models_hist):
        
        for idx, layer_weights in enumerate(new_model.parameters()):

            contribution=client_hist[idx].data*weights[k]
            layer_weights.data.add_(contribution)
            
    return new_model
    

In [33]:
from torch.optim import lr_scheduler

def FedProx(model, training_sets:list, n_iter:int, testing_sets:list, mu=0, 
    file_name="test", epochs=5, lr=10**-2):
    """ all the clients are considered in this implementation of FedProx
    Parameters:
        - `model`: common structure used by the clients and the server
        - `training_sets`: list of the training sets. At each index is the 
            training set of client "index"
        - `n_iter`: number of iterations the server will run
        - `testing_set`: list of the testing sets. If [], then the testing
            accuracy is not computed
        - `mu`: regularization term for FedProx. mu=0 for FedAvg
        - `epochs`: number of epochs each client is running
        - `lr`: learning rate of the optimizer
        - `decay`: to change the learning rate at each iteration
    
    returns :
        - `model`: the final global model 
    """
        
    loss_f=loss_classifier
    
    #Variables initialization
    K=len(training_sets) #number of clients
    n_samples=sum([len(db.dataset) for db in training_sets])
    weights=([len(db.dataset)/n_samples for db in training_sets])    
    
    loss_hist=[[float(loss_dataset(model, dl, loss_f).detach()) 
        for dl in training_sets]]
    acc_hist=[[accuracy_dataset(model, dl) for dl in testing_sets]]
    server_hist=[[tens_param.detach().cpu().numpy()
        for tens_param in list(model.parameters())]]
    models_hist = []
    
    
    server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
    server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])
    print(f'====> i: 0 Loss: {server_loss} Server Test Accuracy: {server_acc}')
    server_loss_list=[]
    server_accuracy_list=[]  
    for i in range(n_iter):
        
        clients_params=[]
        clients_models=[]
        clients_losses=[]
        for k in range(K):
            local_model=deepcopy(model)
            local_optimizer=optim.SGD(local_model.parameters(), lr=0.001, momentum=0.9)
            exp_lr_scheduler = lr_scheduler.StepLR(local_optimizer, step_size=7, gamma=0.1)

            local_loss=local_learning(local_model,mu,local_optimizer,exp_lr_scheduler,
                training_sets[k],epochs,loss_f)
            
            clients_losses.append(local_loss)
                
            #GET THE PARAMETER TENSORS OF THE MODEL
            list_params=list(local_model.parameters())
            list_params=[tens_param.detach() for tens_param in list_params]
            clients_params.append(list_params)    
            clients_models.append(deepcopy(local_model))

            print(f"{k}---local_loss--- {local_loss}" )
        
        
        #CREATE THE NEW GLOBAL MODEL
        model = average_models(deepcopy(model), clients_params, 
            weights=weights)
        models_hist.append(clients_models)
        
        #COMPUTE THE LOSS/ACCURACY OF THE DIFFERENT CLIENTS WITH THE NEW MODEL
        loss_hist+=[[float(loss_dataset(model, dl, loss_f).detach()) 
            for dl in training_sets]]
        acc_hist+=[[accuracy_dataset(model, dl) for dl in testing_sets]]

        server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
        server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])

        print(f'====> i: {i+1} Loss: {server_loss} Server Test Accuracy: {server_acc}')
        server_accuracy_list.append(server_acc)
        server_loss_list.append(server_loss)        

        server_hist.append([tens_param.detach().cpu().numpy() 
            for tens_param in list(model.parameters())])
        
        #DECREASING THE LEARNING RATE AT EACH SERVER ITERATION
            
    return model, loss_hist, acc_hist,server_accuracy_list,server_loss_list


In [34]:
n_iter=40

model_f, loss_hist_FA_iid, acc_hist_FA_iid,server_accuracy_list,server_loss_list = FedProx( model, 
    mnist_iid_train_dls, n_iter, mnist_iid_test_dls, epochs =3)

def plot_acc_loss(title:str, loss_hist:list, acc_hist:list):
    plt.figure()
    
    plt.suptitle(title)

    plt.subplot(1,2,1)
    lines=plt.plot(loss_hist)
    plt.title("Loss")
    plt.legend(lines,["C1", "C2", "C3","C4", "C5", "C6","C7", "C8", "C9","C10"])

    plt.subplot(1,2,2)
    lines=plt.plot(acc_hist )
    plt.title("Accuracy")
    plt.legend(lines,["C1", "C2", "C3","C4", "C5", "C6","C7", "C8", "C9","C10"])
    


import pickle


with open('acc-fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(acc_hist_FA_iid, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('loss-fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(loss_hist_FA_iid, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('server_acc_hist_fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(server_accuracy_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('server_loss_list_fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(server_loss_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
plot_acc_loss("FedAvg MNIST-AVOA-IID-Epoch-5-Client-10", loss_hist_FA_iid, acc_hist_FA_iid)


====> i: 0 Loss: 2.6269259929656985 Server Test Accuracy: 8.3
0---local_loss--- 2.562587261199951
1---local_loss--- 2.5003514289855957
2---local_loss--- 2.600984573364258
3---local_loss--- 2.5416371822357178
4---local_loss--- 2.7303996086120605
5---local_loss--- 2.630730628967285
6---local_loss--- 2.329578399658203
7---local_loss--- 2.467646837234497
8---local_loss--- 2.567526340484619
9---local_loss--- 2.5182137489318848
====> i: 1 Loss: 1.7743332862854007 Server Test Accuracy: 36.6
0---local_loss--- 2.36006498336792
1---local_loss--- 2.518728733062744
2---local_loss--- 2.2925853729248047
3---local_loss--- 2.3917932510375977
4---local_loss--- 2.5103611946105957
5---local_loss--- 2.417584180831909
6---local_loss--- 2.3293988704681396
7---local_loss--- 2.2155792713165283
8---local_loss--- 2.260204792022705
9---local_loss--- 2.247070074081421
====> i: 2 Loss: 1.6482928156852723 Server Test Accuracy: 40.800000000000004
0---local_loss--- 2.3391921520233154
1---local_loss--- 2.5764153003692